In [25]:
import numpy as np
import skfuzzy as fuzz
import pandas as pd
import torch
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler





In [26]:
#Acesso ao dataset
music = pd.read_csv('dataset.csv', index_col=0) #chama o ficheiro e remove coluna desnecessária (index_col)

#Garantir que o acesso foi bem sucedido (faz print das primeiras 5 linhas do dataset)
print(music.head())
music.shape

                 track_id                 artists  \
0  5SuOikwiRyPMVoIQDJUgSV             Gen Hoshino   
1  4qPNDBW1i3p13qLCt0Ki3A            Ben Woodward   
2  1iJBSr7s7jYXzM8EGcbK5b  Ingrid Michaelson;ZAYN   
3  6lfxq3CG4xtTiEg7opyCyx            Kina Grannis   
4  5vjLSffimiIP26QG5WcN2K        Chord Overstreet   

                                          album_name  \
0                                             Comedy   
1                                   Ghost (Acoustic)   
2                                     To Begin Again   
3  Crazy Rich Asians (Original Motion Picture Sou...   
4                                            Hold On   

                   track_name  popularity  duration_ms  explicit  \
0                      Comedy          73       230666     False   
1            Ghost - Acoustic          55       149610     False   
2              To Begin Again          57       210826     False   
3  Can't Help Falling In Love          71       201933     False   
4   

(114000, 20)

In [27]:
##limpeza do dataset
# Contar quantas linhas duplicadas existem (mesmo nome + mesmo artista)
duplicadas_antes = music.duplicated(subset=["track_name", "artists"]).sum()
print(f"Antes da limpeza: {duplicadas_antes} músicas duplicadas")

# Remover duplicados (mesmo nome e mesmo artista)
music = music.drop_duplicates(subset=["track_name", "artists"], keep="first")

#valores em falta por coluna
print(music.isnull().sum()) 
music = music.dropna()  #por serem poucos


Antes da limpeza: 32656 músicas duplicadas
track_id            0
artists             1
album_name          1
track_name          1
popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
dtype: int64


In [33]:
#defenir as features
f0 = music['danceability'].values   #[0,1]
f1 = music['energy'].values         #[0,1]
f2 = music['key'].values            #[-1,11]
f3 = music['loudness'].values       #dB
f4 = music['mode'].values           #binary
f5 = music['acousticness'].values   #[0,1]
f6 = music['tempo'].values          #bpm
f7 = music['time_signature'].values #[0,5]
f8 = music['track_genre'].values    #classes

y = music.values[:, 16]    #valence-target
#COVERTER NOMES EM NUMEROOSSSS
'------------------------------------'



# matriz features
x = music.values[:, [7, 8, 9, 10, 11, 13, 17, 18, 19]]
#x = music[['danceability', 'energy', 'key',.....]]    
'-------------------------------------melhor metodo?'

print(f1)
print(f8)
print(y)
print(x)
music.shape

[0.461 0.166 0.359 ... 0.329 0.506 0.487]
['acoustic' 'acoustic' 'acoustic' ... 'world-music' 'world-music'
 'world-music']
[0.715 0.267 0.12 ... 0.743 0.413 0.708]
[[0.676 0.461 1 ... 87.917 4 'acoustic']
 [0.42 0.166 1 ... 77.489 4 'acoustic']
 [0.438 0.359 0 ... 76.332 4 'acoustic']
 ...
 [0.629 0.329 0 ... 132.378 4 'world-music']
 [0.587 0.506 7 ... 135.96 4 'world-music']
 [0.526 0.487 1 ... 79.198 4 'world-music']]


(81343, 20)

In [29]:
#verificaçaõ da limpeza
duplicadas_depois = music.duplicated(subset=["track_name", "artists"]).sum()
print(f"Depois da limpeza: {duplicadas_depois} músicas duplicadas")


Depois da limpeza: 0 músicas duplicadas


In [ ]:
#train, validation e test split
test_size = 0.3
val_size = 0.5   #será metade dos dados de teste
#Seprarar o conjunto de treino (70%) e o temporário (30%)
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size = test_size, random_state = 42)
#Separar o conjunto temporário em teste (15%) e validação (15%)
x_val, x_test, y_val, y_test = train_test_split(x, y, test_size = val_size, RandomState = 42)

'test será o conjunto nunca visto pelo algoritmo até ao fim'

'-------------------------------------------------'

In [ ]:
#Normalizar varivaeis Key, loudness, tempo, time_signature
#scaler = StandardScaler()
#x_train[ :, [ 2, 3, 6, 7]] = scaler.fit_transform(x_train[ :, [ 2, 3, 6, 7]])
#x_val[ :, [ 2, 3, 6, 7]] = scaler.transform(x_val[ :, 2, 3, 6, 7])

print(x_train)

'''----------PORQUE NÃO POSSO FAZER A NORMALIZAÇÃO ANTES DE SEPARAR ENTRE TESTE E TREINO
OU SEJA DÁ PARA O FAZER SEM LEVAR A OVER FITTING?'''

[[0.728 0.234 7 ... 154.013 4 'chill']
 [0.724 0.759 9 ... 101.943 4 'edm']
 [0.523 0.26 2 ... 110.512 4 'romance']
 ...
 [0.487 0.398 5 ... 109.389 3 'tango']
 [0.564 0.731 9 ... 95.036 4 'acoustic']
 [0.676 0.951 9 ... 85.72 4 'comedy']]


'----------PORQUE NÃO POSSO FAZER A NORMALIZAÇÃO ANTES DE SEPARAR ENTRE TESTE E TREINO\nOU SEJA DÁ PARA O FAZER SEM LEVAR A OVER FITTING?'

In [34]:
#Normalizar varivaeis Key, loudness, tempo, time_signature
scaler = MinMaxScaler()
x_train[ :, [ 2, 3, 6, 7]] = scaler.fit_transform(x_train[ :, [ 2, 3, 6, 7]])
#x_val[ :, [ 2, 3, 6, 7]] = scaler.transform(x_val[ :, 2, 3, 6, 7])

print(x_train)

[[0.728 0.234 0.6363636363636364 ... 0.6918667595067497 0.8 'chill']
 [0.724 0.759 0.8181818181818182 ... 0.45795467307562726 0.8 'edm']
 [0.523 0.26 0.18181818181818182 ... 0.4964488668268907 0.8 'romance']
 ...
 [0.487 0.398 0.4545454545454546 ... 0.49140405651265695
  0.6000000000000001 'tango']
 [0.564 0.731 0.8181818181818182 ... 0.4269266188989466 0.8 'acoustic']
 [0.676 0.951 0.8181818181818182 ... 0.3850767053749916 0.8 'comedy']]
